In [1]:
from model.model import ConvModel, SimCLR
import argparse
import torch
import torch.backends.cudnn as cudnn
from torchvision import models
from torchvision import transforms, datasets


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
    gpu_index = 0
else:
    device = torch.device('cpu')
    gpu_index = -1


In [3]:
color_jitter = transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)

composed_transforms = transforms.Compose([transforms.RandomResizedCrop(size=32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomApply([color_jitter], p=0.8),
                                      transforms.RandomGrayscale(p=0.2),
                                      transforms.GaussianBlur(kernel_size=3),
                                    transforms.ToTensor()],
                                    )
# composed_transforms = transforms.Compose([
#         transforms.GaussianBlur(kernel_size=3),
#       transforms.RandomRotation(degrees=10), # TODO
#       transforms.RandomGrayscale(p=0.2),
#       transforms.ToTensor()])

In [4]:
train_dataset_1 = datasets.CIFAR10('./datasets', train=True,
                               transform=composed_transforms,
                               download=True)
train_dataset_2 = datasets.CIFAR10('./datasets', download=True, train=True, transform=composed_transforms)
g = torch.Generator()
g.manual_seed(0)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader_1 = torch.utils.data.DataLoader(
    train_dataset_1, batch_size=256, shuffle=False,
     pin_memory=True, drop_last=True, generator=g)
train_loader_2 = torch.utils.data.DataLoader(train_dataset_2, batch_size=256, shuffle=True, pin_memory=True, drop_last=True, generator=g)

In [9]:
model = ConvModel(out_dim=10, dataset='CIFAR10')

model.load_state_dict(torch.load('saved_models/model_final_cifar10_200_epochs.pth'))

<All keys matched successfully>

In [7]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader_1), eta_min=0,
                                                       last_epoch=-1)

with torch.cuda.device(gpu_index):
    simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100)
    simclr.train(train_loader_1, shuffle=True)

  0%|          | 0/195 [00:00<?, ?it/s]C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torch\optim\lr_scheduler.py:807: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
 44%|████▎     | 85/195 [00:31<00:40,  2.71it/s]


KeyboardInterrupt: 

In [8]:
torch.save(model.state_dict(), 'saved_models/model_final_cifar10_13_12_2022.pth')

In [10]:
model.backbone.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=512, out_features=128, bias=True),
    torch.nn.ReLU(inplace=True),
    # torch.nn.BatchNorm1d(128),
    torch.nn.Linear(in_features=128, out_features=10, bias=True),
)


In [11]:
model.backbone.requires_grad_(False)
model.backbone.fc.requires_grad_(True)

for parameter in model.backbone.named_parameters():
    print(f'{parameter[0]}: {parameter[1].requires_grad}')


conv1.weight: False
bn1.weight: False
bn1.bias: False
layer1.0.conv1.weight: False
layer1.0.bn1.weight: False
layer1.0.bn1.bias: False
layer1.0.conv2.weight: False
layer1.0.bn2.weight: False
layer1.0.bn2.bias: False
layer1.1.conv1.weight: False
layer1.1.bn1.weight: False
layer1.1.bn1.bias: False
layer1.1.conv2.weight: False
layer1.1.bn2.weight: False
layer1.1.bn2.bias: False
layer2.0.conv1.weight: False
layer2.0.bn1.weight: False
layer2.0.bn1.bias: False
layer2.0.conv2.weight: False
layer2.0.bn2.weight: False
layer2.0.bn2.bias: False
layer2.0.downsample.0.weight: False
layer2.0.downsample.1.weight: False
layer2.0.downsample.1.bias: False
layer2.1.conv1.weight: False
layer2.1.bn1.weight: False
layer2.1.bn1.bias: False
layer2.1.conv2.weight: False
layer2.1.bn2.weight: False
layer2.1.bn2.bias: False
layer3.0.conv1.weight: False
layer3.0.bn1.weight: False
layer3.0.bn1.bias: False
layer3.0.conv2.weight: False
layer3.0.bn2.weight: False
layer3.0.bn2.bias: False
layer3.0.downsample.0.weight: 

In [12]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader_2), eta_min=0,
                                                       last_epoch=-1)

# Glowa musi uczyc sie w sposob nadzorowany
with torch.cuda.device(gpu_index):
    simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100)
    simclr.train_head(train_loader_2)


  0%|          | 0/195 [00:01<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [10]:
test_dataset = datasets.CIFAR10('./datasets', download=True, train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=True, pin_memory=True, drop_last=True, generator=g)

Files already downloaded and verified


In [14]:
import logging
from torch.utils.tensorboard import SummaryWriter
import os
from model.model import accuracy
from tqdm import tqdm

total = 0
running_accuracy = 0
n_iter = 0
writer = SummaryWriter()
logging.basicConfig(filename=os.path.join(writer.log_dir, 'training.log'), level=logging.DEBUG)
criterion = torch.nn.CrossEntropyLoss().to(device)
model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        top1, top5 = accuracy(output, labels, topk=(1, 5))
        writer.add_scalar('loss', loss, global_step=n_iter)
        writer.add_scalar('acc/top1', top1[0], global_step=n_iter)
        writer.add_scalar('acc/top5', top5[0], global_step=n_iter)
        n_iter += 1


100%|██████████| 39/39 [00:02<00:00, 18.29it/s]
